In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [17]:
model = keras.models.load_model('./Trained Models/m1.h5')
url = './Testing/test2.mp4'
# url = '/home/avishrant/Downloads/test.mp4'

In [24]:
import cv2
class Draw(object):
    def __init__(self):
        pass

    def set_ref(self, frame):
        self.ref = frame
    
    def drawfunc(self, frame):
        diff = cv2.absdiff(self.ref, frame)
        gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5,5), 0)
        _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
        dilated = cv2.dilate(thresh, None, iterations=3)
        contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        self.ref = frame.copy()
        for contour in contours:
            (x, y, w, h) = cv2.boundingRect(contour)
            if cv2.contourArea(contour) < 1500:
                continue
            frame = cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        return frame

In [25]:
# Live Generator
vid = cv2.VideoCapture(url)
font = cv2.FONT_HERSHEY_SIMPLEX 
org = (10, 40) 
fontScale = 0.7
thickness = 1
f_stat = False
obj = Draw()
while (cv2.waitKey(1) == -1):
    ret, frame = vid.read()
    
    if not ret:
        break
    tmp = cv2.resize(frame , (150, 150))
    tmp = tmp / 255.0
    pred = model.predict(np.array([tmp]))
    if not f_stat:
        f_stat = True
        obj.set_ref(frame) 
    else:
        frame = obj.drawfunc(frame)

    string = "Suspicious" if pred[0][0] < 0.5 else "Peaceful"
    string += f" {str(pred[0][0])}"
    color = (0, 0, 255) if pred[0][0] < 0.5 else (255, 0, 0)
    frame = cv2.putText(frame, string, org, font, fontScale, color, thickness, cv2.LINE_AA) 
    cv2.imshow("Video", frame)
    
vid.release()
cv2.destroyAllWindows()

In [23]:
# Video Generator
THRESH = 0.5
F_GROUPS = 5

import cv2
import numpy as np
font = cv2.FONT_HERSHEY_SIMPLEX 
org = (10, 40) 
fontScale = 0.7
thickness = 1

W, H = None, None
out = None      
vid = cv2.VideoCapture(url)
orig_pred = []
fcount = 0
fourcc = cv2.VideoWriter_fourcc('M','J','P','G')

while (cv2.waitKey(1) == -1):
    ret, frame = vid.read()
    if not ret:
        break
    if W is None or H is None:
        (H, W) = frame.shape[:2]
        
    tmp = cv2.resize(frame, (150, 150))
    tmp = tmp / 255.0
    if not fcount % 2:
        pred = model.predict(np.array([tmp]))
        orig_pred.append(pred[0][0])
    else:
        pred = orig_pred[-1]
        orig_pred.append(pred)

    if fcount >= F_GROUPS:
        roll = np.array(orig_pred[fcount-F_GROUPS:fcount]).mean(axis=0)
        string = "Suspicious" if roll < THRESH else "Peaceful"
        color = (0, 0, 255) if roll < THRESH else (255, 0, 0)
    else:
        string = "Calibrating"
        color = (0, 255, 0)
    
    frame = cv2.putText(frame, string, org, font, fontScale, color, thickness, cv2.LINE_AA) 
    if out is None:
        out = cv2.VideoWriter("output.avi", fourcc, 15, (W, H))

    fcount += 1
    out.write(frame)
    cv2.imshow("Frame", frame)

print("Done !")
vid.release()
out.release()
cv2.destroyAllWindows()

Done !
